# "DHG"
> [Deep Hyper Graph](https://deephypergraph.com/#/)

- toc: true
- branch: master
- badges: true
- comments: true
- keywords : Library, Graph, HyperGraph
- categories: [Graph, HyperGraph]
- permalink: /deep-hyper-graph/

### installation guide
- [docs/install](https://deephypergraph.readthedocs.io/en/0.9.3/start/install.html)


```py
# conda env : python 3.8
$ pip install dhg

# dependencies / requirements
$ conda install -c pytorch pytorch    # (최신버전 설치) dhg는 torch를 확장한 패키지, (23.03 현재 m1 가속을 지원한다.)
```

In [1]:
# for M1 MAC / pytorch check

import torch; device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

print(f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS supportive build? : {torch.backends.mps.is_built()}")
print(f"MPS available : {torch.backends.mps.is_available()}")
!python -c 'import platform;print(platform.platform())'

PyTorch version:2.0.0
MPS supportive build? : True
MPS available : True
macOS-13.0-arm64-arm-64bit


### Cooking200 Example

> The Cooking 200 dataset (dhg.data.Cooking200) is collected from Yummly.com for vertex classification task. It is a hypergraph dataset, in which vertex denotes the dish and hyperedge denotes the ingredient. Each dish is also associated with category information, which indicates the dish’s cuisine like Chinese, Japanese, French, and Russian.

> Kaggle [https://www.kaggle.com/c/whats-cooking](https://www.kaggle.com/c/whats-cooking) 에서 공개된 데이터셋이다.
> CLI환경에서는 `kaggle competitions download -c whats-cooking` 으로 다운로드 가능하며 train.json은 id:int, cuisine:int,ingredients:list 구조로 구성되어있다.

graph dataset은 cora, pubmed 등 논문과 논문의 저자 관계를 다룬 데이터셋을 사용하는게 일반적이지만 hypergraph dataset은 그보다 조금 더 복잡한, 다수의 관계를 매핑한 데이터셋을 사용한다. `Yummly` 에서 제작한 데이터셋인 Cooking200의 구조는 아래와 같다.
- vertex (=node) : dish, receipt
- hyperedge (node 간의 관계) : ingredient
    - hyperedge: A connection between two or more vertices of a hypergraph. A hyperedge connecting just two vertices is simply a usual graph edge.
    - G는 결국 HG의 특수한 형태로, hyperedge를 edge의 확장으로 생각하는 것 보다 hyperedge의 특수한 경우를 edge로 이해하는 편이 빠르다. 결국 **edge는 node간의 관계를 의미하는 용어**이고 일반 graph에서 edge는 1:1 관계, hypergraph에서의 edge, 즉 hyperedge는 다수간의 연결을 의미한다.

Cooking200은 크게 요리(dish), 분류(category), 재료(ingredient) 로 구분되는데, 각 재료는 한가지 요리에 종속되지 않으며 

#### NOTE
1. The dataset is a hypergraph dataset, which cannot be directly used for GCN model. Thus, `the clique expansion is adpoted` to reduce the hypergraph structure to a graph structure.
    - GCN 구조상 A를 만들지 않고서는 GCN에 hypergraph dataset을 사용할 수 없다.
    - expansion의 경우 `dhg` 에서 제공하는 기능으로 구현할 수 있는데 이는 다음 [공식문서](https://deephypergraph.readthedocs.io/en/latest/tutorial/structure.html#reduced-from-high-order-structures)를 참고 할 것. star expansion, clique expansion, hypergcn* 의 방법을 지원한다.
        - *hypergcn : ('In the HyperGCN paper, the authors also describe a method to reduce the hyperedges in the hypergraph to the edges in the graph as the following figure.')
        ![hypergraph from hypergcn](https://deephypergraph.readthedocs.io/en/latest/_images/hypergcn.png)
2. The dataset `do not contain the vertex features`. Thus, we generate a identity matrix for vertex features.

In [2]:
# Example : https://deephypergraph.readthedocs.io/en/latest/examples/vertex_cls/hypergraph.html#gcn-on-cooking200

'''Import libraries'''

import time
from copy import deepcopy

import torch
import torch.optim as optim
import torch.nn.functional as F

from dhg import Graph, Hypergraph
from dhg.data import Cooking200
from dhg.models import GCN
from dhg.random import set_seed
from dhg.metrics import HypergraphVertexClassificationEvaluator as Evaluator

'''Define Functions'''

def train(net, X, A, lbls, train_idx, optimizer, epoch):
    net.train()

    st = time.time()
    optimizer.zero_grad()
    outs = net(X, A)
    outs, lbls = outs[train_idx], lbls[train_idx]
    loss = F.cross_entropy(outs, lbls)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Time: {time.time()-st:.5f}s, Loss: {loss.item():.5f}")
    return loss.item()


@torch.no_grad()
def infer(net, X, A, lbls, idx, test=False):
    net.eval()
    outs = net(X, A)
    outs, lbls = outs[idx], lbls[idx]
    if not test:
        res = evaluator.validate(lbls, outs)
    else:
        res = evaluator.test(lbls, outs)
    return res

'''Main'''
if __name__ == "__main__":
    set_seed(2021)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    evaluator = Evaluator(["accuracy", "f1_score", {"f1_score": {"average": "micro"}}])
    data = Cooking200()

    X, lbl = torch.eye(data["num_vertices"]), data["labels"]
    ft_dim = X.shape[1]
    HG = Hypergraph(data["num_vertices"], data["edge_list"])
    G = Graph.from_hypergraph_clique(HG, weighted=True)
    train_mask = data["train_mask"]
    val_mask = data["val_mask"]
    test_mask = data["test_mask"]

    net = GCN(ft_dim, 32, data["num_classes"], use_bn=True)
    optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

    X, lbl = X.to(device), lbl.to(device)
    G = G.to(device)
    net = net.to(device)

    best_state = None
    best_epoch, best_val = 0, 0
    for epoch in range(3):        # 200 epoch -> 3 으로 줄여서 테스트
        # train
        train(net, X, G, lbl, train_mask, optimizer, epoch)
        # validation
        if epoch % 1 == 0:
            with torch.no_grad():
                val_res = infer(net, X, G, lbl, val_mask)
            if val_res > best_val:
                print(f"update best: {val_res:.5f}")
                best_epoch = epoch
                best_val = val_res
                best_state = deepcopy(net.state_dict())
    print("\ntrain finished!")
    print(f"best val: {best_val:.5f}")
    # test
    print("test...")
    net.load_state_dict(best_state)
    res = infer(net, X, G, lbl, test_mask, test=True)
    print(f"final result: epoch: {best_epoch}")
    print(res)

/opt/homebrew/Caskroom/miniforge/base/envs/hypergraph-py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/hypergraph-py38/lib/python3.8/site-packages/dhg/structure/graphs/graph.py:247: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1678455016227/work/aten/src/ATen/SparseCsrTensorImpl.cpp:56.)
  adj = miu * hypergraph.H.mm(hypergraph.H_T).coalesce().cpu().clone()


Epoch: 0, Time: 7.83699s, Loss: 3.02474
update best: 0.05000
Epoch: 1, Time: 0.22393s, Loss: 2.47472
Epoch: 2, Time: 0.21954s, Loss: 2.41068

train finished!
best val: 0.05000
test...
final result: epoch: 0
{'accuracy': 0.01942024938762188, 'f1_score': 0.0019050287155063736, 'f1_score -> average@micro': 0.019420248464943595}


### API structures & references

- docs 참고, 주요 구조 기록